In [9]:
import os
import shutil
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab import files
import kaggle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Model


In [8]:
os.environ['KAGGLE_USERNAME'] = 'nitinreddyrm'
os.environ['KAGGLE_KEY'] = '810fdc8e3dd73a3b516939692c7ca5e8'

In [11]:
kaggle.api.authenticate()

In [12]:
!mkdir ~/.kaggle

In [13]:
#Downloading the data from kaggle website
!kaggle competitions download -c dog-breed-identification

 99% 682M/691M [00:11<00:00, 48.8MB/s]
100% 691M/691M [00:11<00:00, 64.6MB/s]


In [14]:
import zipfile

In [15]:
def unzip_file(zip_file_path, extract_path):
  """Unzips a ZIP file to the specified directory.

  Args:
    zip_file_path: The path to the ZIP file.
    extract_path: The path to the directory where the files should be extracted.
  """

  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Example usage
zip_file_path = "/content/dog-breed-identification.zip"
extract_path = "/content/dog-breed-identification"

unzip_file(zip_file_path, extract_path)

In [16]:
data=cv2.imread('content/dog-breed-identification/train/')

In [17]:
labels=pd.read_csv('/content/dog-breed-identification/labels.csv')

In [18]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [19]:
labels['File_path']='/content/dog-breed-identification/train/'+labels.id+'.jpg'

In [20]:
labels.breed.nunique()

120

In [21]:
labels.head()

,id,breed,File_path
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull,/content/dog-breed-identification/train/000bec...
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo,/content/dog-breed-identification/train/001513...
2,001cdf01b096e06d78e9e5112d419397,pekinese,/content/dog-breed-identification/train/001cdf...
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick,/content/dog-breed-identification/train/00214f...
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever,/content/dog-breed-identification/train/0021f9...


In [22]:
labels.File_path.iloc[0]

'/content/dog-breed-identification/train/000bec180eb18c7604dcecc8fe0dba07.jpg'

In [23]:
breeds={}
for i in labels.breed:
  breeds[i]=labels[labels['breed']==i].File_path

In [24]:
import os
import shutil

In [25]:
for i in breeds.keys():
  file_path='content/dog-breed-identification/train/'+i
  os.makedirs(file_path)
  for j in breeds[i]:
    shutil.move(j,file_path)

In [26]:
traning_datagen=ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True
                                  )

In [27]:
traning_set=traning_datagen.flow_from_directory('/content/content/dog-breed-identification/train',
                                                target_size=(224,224),
                                                batch_size=32,
                                                class_mode='categorical')

Found 10222 images belonging to 120 classes.


In [28]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True)

In [29]:
from tensorflow.keras.applications.resnet import ResNet50

In [30]:
image_size=[224,224]
ResNet=ResNet50(input_shape= image_size + [3], weights='imagenet' , include_top=False)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [31]:
for layers in ResNet.layers:
  layers.trainable=False

In [32]:
resnet_flatten=keras.layers.Flatten()(ResNet.output)

In [33]:
res_output_layer=keras.layers.Dense(120,activation='softmax')(resnet_flatten)

In [34]:
resnet_model= Model(inputs=ResNet.input,outputs=res_output_layer)

In [35]:
resnet_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 35,630,072 (135.92 MB)

 Trainable params: 12,042,360 (45.94 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [36]:
resnet_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
resnet_model.fit(traning_set,epochs=20)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 72/320 ━━━━━━━━━━━━━━━━━━━━ 23:35 6s/step - accuracy: 0.0087 - loss: 28.9063

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB3

In [ ]:
efficientnetb3=EfficientNetB3(input_shape=image_size+[3], weights='imagenet', include_top=False)

43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
for layers in efficientnetb3.layers:
  layers.trainable=False

In [ ]:
efficient_flatten=keras.layers.Flatten()(efficientnetb3.output)

In [ ]:
efficient_ouput_layer=keras.layers.Dense(120,activation='softmax')(efficient_flatten)

In [ ]:
efficient_modelB3=Model(inputs=efficientnetb3.input,outputs=efficient_ouput_layer)

In [ ]:
efficient_modelB3.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling (Rescaling)     │ (None, 224, 224, 3)    │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization             │ (None, 224, 224, 3)    │              7 │ rescaling[0][0]        │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_1 (Rescaling)   │ (None, 224, 224, 3)    │              0 │ normalization[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, 225, 225, 3)    │              0 │ rescaling_1[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, 112, 112, 40)   │          1,080 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 112, 112, 40)   │            160 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, 112, 112, 40)   │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, 112, 112, 40)   │            360 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, 112, 112, 40)   │            160 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, 112, 112, 40)   │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 40)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 40)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 10)       │            410 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 19,815,335 (75.59 MB)

 Trainable params: 9,031,800 (34.45 MB)

 Non-trainable params: 10,783,535 (41.14 MB)

In [ ]:
efficient_modelB3.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

387

In [ ]:
efficient_modelB3.fit(traning_set,epochs=20)

NameError: name 'efficient_modelB3' is not defined